1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


hit_rate@k - был "куплен" товар из k релевантных товаров, precision@k - сколько из предложенных k позиций были купленными
hit_rate@k - предподчительно использовать когда есть набор релевантных товаров, например при рекомендации пользовотелю новостей по ключевым словам
precision@k - например рекомендация приобретения товаров из списка "с этим товаром часто покупают"


2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Как Вы думаете, почему именно логарифм? Логарифм растет постепенно. 
Какую функцию можно использовать вместо логарифма?  просто делить на i
Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм. - Log Loss логистической функции ошибки


3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

ERR - метрика используетсяпри моделировании действий пользователя в поисковых системах.
Модель которая предполагает единственный документ удовлетворяющий результату поиска. 
Предполагается, что пользователь поочередно просматривает ранжированны результат поиска
и отмечает удовлетворяет ли документ поиску, если да, то поиск останавливается.

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [1]:
import numpy as np

boughted = [1, 3, 5, 7, 9, 11] 
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 
#pr@8 = (0+0+1+1+1+1+0+0)/(1+1+1+1+1+1+1+1) = 50%

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision
print(f"pr@8 = {precision_at_k(recommended, boughted, 8)}")

pr@8 = 0.5


In [2]:
#rec@8 = (0+0+1+1+1+1+0+0)/6= 66,66%

def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall
print(f"rec@8 = {recall_at_k(recommended, boughted, 8)}")

rec@8 = 0.6666666666666666


In [3]:
#AP@8 = 1/2+2/3+2/5+4/6 = (8/5)/10*100 = 24%

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i-1] == True: # в уроке стоит просто i, не поняла почему? массив начинается с 0
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result
print(f"AP@8 = {ap_k(recommended, boughted, 8)}")


AP@8 = 0.24333333333333332


In [4]:
boughted = [1, 3, 5, 7, 9, 11] 
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 
#NDCG@8 = 1/8∗(0/1+1/𝑙𝑜𝑔(2)+1/𝑙𝑜𝑔(3)+0/𝑙𝑜𝑔(4)+1/𝑙𝑜𝑔(5)+1/log(6)+0/log(7)+0/log(8))/
#         1/8∗(1/1+1/𝑙𝑜𝑔(2)+1/𝑙𝑜𝑔(3)+1/𝑙𝑜𝑔(4)+1/𝑙𝑜𝑔(5)+1/log(6)+1/log(7)+1/log(8))  = 0,52
import math

def dcg_k(recommended_list, bought_list, k=5):
   
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(recommended_list,bought_list)
    
    if sum(flags) == 0 or k==0:
        return 0
        
    summary = 0
    
    for i in range(1,k+1):
        if flags[i-1]:
            summary += 1/(1 if i==1 else math.log(i,2) )  
    
    return summary/k
                          
print(f"dcg@8 = {dcg_k(recommended, boughted, 8)}")
    
def ndcg_k(recommended_list, bought_list, k=5, kideal=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list_ideal = np.array(recommended_list[:kideal])
    
    dcg = dcg_k(recommended_list, bought_list, k)
    dcg_ideal = dcg_k(recommended_list, bought_list_ideal, kideal)
    
    if dcg_ideal==0:
        return 0
    
    return dcg/dcg_ideal

print(f"ndcg@8 = {ndcg_k(recommended, boughted, 8,8)}")   
    

dcg@8 = 0.306057389859924
ndcg@8 = 0.527912744563727


In [5]:
#MRR@8
def reciprocal_rank(recommended_list, bought_list,k=5):
    ranks=0.
    for i, item_rec in enumerate(recommended_list[:k]):
        for item_bought in bought_list[:min(len(bought_list),k)]:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

print(f"MRR@8 = {reciprocal_rank(recommended, boughted,8)}")   


MRR@8 = 0.12
